
<br>
The following script has 4 main sections.<br>
We first calculate the optimal investment weights of a trading strategy<br>
by optimization. Then, we calculate the in and out of sample returns<br>
for the strategy and the chosen benchmark.<br>
Ultimately, we calculate performance and risk ratios and plot portfolio<br>
characteristics of the investment strategy in comparison to the benchmark.<br>


oad libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from scipy.optimize import minimize
from scipy.optimize import Bounds
import datetime as dt
from sklearn.linear_model import LinearRegression
import dataframe_image as dfi

###########################################################################<br>
                                                                          #<br>
                               #SECTION 1                                 #<br>
      Define key parameters, import needed files and define investment   #<br>
      universe of stocks which the strategy invests in                    #<br>
                                                                          #<br>
###########################################################################

efine parameters of investment period start

In [ ]:
start_backtesting = np.datetime64("2011-01-01")
end_backtesting = np.datetime64("2021-12-31")

mport needed CSV files

In [ ]:
stocks_index = pd.read_csv("files/index_constituents_data.csv", index_col= 0)
benchmark = pd.read_csv("files/benchmark.csv", index_col= 0)
Gross_Price = pd.read_csv("files/Gross_Prices_EUR.csv", index_col= 0)
Net_Price = pd.read_csv("files/Net_Prices_EUR.csv", index_col= 0)

ormat index to date

In [ ]:
Gross_Price.index = pd.to_datetime(Gross_Price.index)
Net_Price.index = pd.to_datetime(Net_Price.index)
benchmark.index = pd.to_datetime(benchmark.index)


<br>
Select stocks which the strategy invests in<br>
our strategy goes long in the top 10 highest dividend stocks and short in the<br>
top 10 lowest dividend stocks --> We select these stocks as our investmnet stocks<br>
drop BHP stock for which dividends have not been calculated correctly!<br>


In [ ]:
stocks_index = stocks_index.sort_values(by = "Yield", ascending= False)
stocks_index = stocks_index.drop("BHP.L")
stocks_invest = stocks_index.iloc[np.r_[0:10, 39:49]]

elect prices of stocks which we invest in

In [ ]:
Gross_Price_selected = Gross_Price[stocks_invest.index]
Net_Price_selected = Net_Price[stocks_invest.index]

###########################################################################<br>
                                                                          #<br>
                               #SECTION 2                                 #<br>
    Calculate Optimal Strategy stock weights with various constraints     #<br>
                      and visualize optimization                          #<br>
                                                                          #<br>
###########################################################################

stimate expected returns and var-cov matrix for optimization (with in sample data!)

In [ ]:
ER = Gross_Price_selected[Gross_Price_selected.index < end_backtesting].pct_change().mean()
S = Gross_Price_selected[Gross_Price_selected.index < end_backtesting].pct_change().cov()

efine functions for optimization (variance, return,  negative sharp ratio)

In [ ]:
def pvar(w, S):
    return (w.T @ S @ w)

In [ ]:
def pret(w, ER):
    return (w.T @ ER)

In [ ]:
def sharpe(w ,ER, S):
    return -(w.T @ ER)/ ((w.T @ S @ w) ** 0.5)

---------------------------------------------------------------------------<br>
Calculate Optimized Portfolio<br>
---------------------------------------------------------------------------

In [ ]:
N = len(ER)
x0 = np.ones(N)/N


<br>
#set up constraints<br>
first constraint -> total investment = 100% long<br>
second constraint- > Values smaller than - min_weight (shocks which are shorted)<br>
third constraint -> Values larger than min_weight (stock which are longed)<br>
bounds = maximum long and short % for stocks<br>


In [ ]:
min_weight = 0.02

In [ ]:
cons_multiple = ({"type": "eq", "fun" : lambda x: np.sum(x) - 1},
                {"type": "ineq", "fun" : lambda x: -min_weight -x[10:20]},
                {"type": "ineq", "fun" : lambda x: x[0:10]- min_weight})

In [ ]:
bounds = Bounds(-0.2, 0.2)

nly constraint for unconstrained portfolio --> Total stock weights == 100%

In [ ]:
cons_simple = ({"type": "eq", "fun" : lambda x: np.sum(x) - 1})

alculate optimized values (GMVP and MSRP constrained and unconstrained)

In [ ]:
GMVP_const = minimize(pvar, x0, method='SLSQP', args=S, constraints = cons_multiple,
                      options={'disp': True, 'ftol': 1e-9}, bounds = bounds)

In [ ]:
GMVP_unconst = minimize(pvar, x0, method = "SLSQP", args = S,constraints = cons_simple,
                        options={'disp': True, 'ftol': 1e-9})

In [ ]:
MSRP_const = minimize(sharpe, x0, method='SLSQP', args=(ER, S), constraints=cons_multiple,
                      options={'disp': True, 'ftol': 1e-9}, bounds = bounds)

In [ ]:
MSRP_unconst = minimize(sharpe, x0, method='SLSQP', args=(ER,S), constraints = cons_simple,
                        options={'disp': True, 'ftol': 1e-9})

omplete data set of selected stocks with calculated constrained MSRP values & export for presentation

In [ ]:
stocks_invest["weights"] = MSRP_const.x
stocks_invest["weights_unconst"] = MSRP_unconst.x
dfi.export(stocks_invest, "plots/selected_portfolio_characteristics.png")

--------------------------------------------------------------------------

In [ ]:
                  #Visualize efficient Frontier
#---------------------------------------------------------------------------


<br>
Generate efficient frontier plot by doing the following: <br>
1. Minimize the negative value of the expected portfolio return <br>
given a deterministic variance and the same rules as above (eg. min 2% in each stock)<br>
as restrictions in the optimization. --> We thus retrieve the maximum return given <br>
all restrictions for a given variance. <br>
2. We construct "random portfolios" which fulfill all the restrictions by minimizing a function<br>
which consists of a random array and a modulus operation which should randomize the optimization <br>
results. We can then calculate the expected return and variance of these portfolios and add them <br>
to the plot.<br>
<br>
alculate volatility and expected return of GMVP and MSRP constrained for plotting further below

In [ ]:
GMVP_const_ER = pret(w = GMVP_const.x, ER = ER) * 250
GMVP_const_VAR = m.sqrt(pvar(w = GMVP_const.x, S = S) * 250)
MSRP_const_ER =  pret(w = MSRP_const.x, ER = ER) * 250
MSRP_const_VAR = m.sqrt(pvar(w = MSRP_const.x, S = S) * 250)

efine negative portfolio return function

In [ ]:
def pret_sim(w, ER):
    return (-(w.T @ ER))

ariances to loop over

In [ ]:
var_list = np.arange(0.1, 0.6, 0.01)
bounds = Bounds(-0.2, 0.2)

nitialize lists to store results in

In [ ]:
MVF_var_const = []
MVF_ret_const = []

oop over variances and maximize expected return given that portfolio variance = given variance

In [ ]:
for i in var_list:
    max_var = i
    cons = ({"type": "eq", "fun" : lambda x: np.sum(x) - 1},
            {"type": "eq", "fun" : lambda x: m.sqrt((x.T @ S @ x)* 250) - max_var},
            {"type": "ineq", "fun": lambda x: -min_weight - x[10:20]},
            {"type": "ineq", "fun": lambda x: x[0:10] - min_weight})
    maximized = minimize(pret_sim, x0, method='SLSQP', args= ER,
                         options={'disp': True, 'ftol': 1e-9},
                         constraints=cons, bounds = bounds)

    #only store result if optimization was successful!
    if maximized.success:
        MVF_var_const.append(i)
        MVF_ret_const.append(maximized.fun * -250)
    else:
        continue

--------------------------------------------------------------------------

In [ ]:
                  #Visualize efficient Frontier
#---------------------------------------------------------------------------

In [ ]:
random_portfolio = []

random" function which when optimized gives "random" portfolios

In [ ]:
def rand_funct(x,y):
    return ((x % y)/y).sum()

eight constraints of individual stocks

In [ ]:
cons = ({"type": "eq", "fun" : lambda x: np.sum(x) - 1},
        {"type": "ineq", "fun": lambda x: -min_weight - x[10:20]},
        {"type": "ineq", "fun": lambda x: x[0:10] - min_weight})

imulate 100 different portfolios which fulfill the constraints

In [ ]:
i = 0
while i < 100:
    y = np.random.uniform(low= 0, high=1, size=N)
    MSRP_sim = minimize(rand_funct, x0, method='SLSQP', args= y,
                        constraints=cons, options={'disp': True, 'ftol': 1e-9},
                        bounds = bounds)
    i = i + 1

    #only store successful optimizations
    if MSRP_sim.success:
        random_portfolio.append(MSRP_sim.x)
    else:
        continue

In [ ]:
MVF_randvar_const = []
MVF_randret_const = []

alculate return and variance (yearly) from calculated "random" portfolios

In [ ]:
for i in range(len(random_portfolio)):
    exp_return = pret(w = random_portfolio[i], ER = ER)
    exp_var = pvar(w = random_portfolio[i], S = S)
    exp_return_year = exp_return * 250
    exp_var_year = m.sqrt(exp_var * 250)
    MVF_randvar_const.append(exp_var_year)
    MVF_randret_const.append(exp_return_year)

lot results

In [ ]:
MVF_var_const.insert(0, GMVP_const_VAR)
MVF_ret_const.insert(0, GMVP_const_ER)
CAL_x = np.linspace(0, MSRP_const_VAR + 0.2, 50, endpoint=True)
SR_MSRP = MSRP_const_ER / MSRP_const_VAR
CAL_y = 0 + SR_MSRP*CAL_x

In [ ]:
frontier_points = pd.DataFrame({"variance": MVF_var_const, "return": MVF_ret_const})
max_ret_index = frontier_points["return"].idxmax()

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))
plt.plot(frontier_points.variance.iloc[0:max_ret_index + 1], frontier_points["return"].iloc[0:max_ret_index + 1])
plt.plot(frontier_points.variance.iloc[max_ret_index: len(frontier_points)],
         frontier_points["return"].iloc[max_ret_index: len(frontier_points)], color = "grey", ls =  "--")
plt.scatter(MVF_randvar_const, MVF_randret_const)
plt.scatter(GMVP_const_VAR, GMVP_const_ER, s = 70)
plt.scatter(MSRP_const_VAR, MSRP_const_ER, s = 70)
plt.plot(CAL_x, CAL_y)
plt.xlim([0,0.4])
plt.ylim([0,0.4])
plt.title("Minimum Variance Frontier Simulated", size = 25)
plt.xlabel("Volatility", size = 15)
plt.ylabel("Return", size = 15)
plt.legend(["Minimum Variance Frontier", "?", "Capital Allocation Line","random portfolios", "GMVP", "MSRP"],
           prop={'size': 10})
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.savefig("minimumvariancefrontier.png")

###########################################################################<br>
                                                                          #<br>
                               #SECTION 2                                 #<br>
                          Calculate  returns of                           #<br>
                strategy and benchmark in and out of sample               #<br>
                                                                          #<br>
###########################################################################

unction which calculates the indexed performance (start = 1) of various stocks and start investment weights

In [ ]:
def indexed_performance(start, prices, weights, end = None):

    #filter desired time range
    if end is None:
        temp = prices[prices.index >= start]
    else:
        temp = prices[(prices.index >= start) & (prices.index < end)]

    #define daily percentage changes + 1
    temp = temp.pct_change() + 1

    #replace first row with weights of each stock --> Initial Investment = 1
    temp.iloc[0] = weights

    #take cumulative return of each stock * initial investment (weight) and sum up horizontaly
    temp = temp.cumprod().sum(axis = 1)
    return(temp)

alculate weights of short stocks / long stocks  that they sum up to 1

In [ ]:
weights_short = stocks_invest.weights[stocks_invest.weights < 0] / stocks_invest.weights[stocks_invest.weights < 0].sum()
weights_long =  stocks_invest.weights[stocks_invest.weights > 0] / stocks_invest.weights[stocks_invest.weights > 0].sum()

alculate needed indexed performance series

In [ ]:
in_sample_net = indexed_performance(start = start_backtesting, end = end_backtesting,
                                    weights = MSRP_const.x, prices = Net_Price_selected)

In [ ]:
in_sample_gross = indexed_performance(start = start_backtesting, end = end_backtesting,
                                    weights = MSRP_const.x, prices = Gross_Price_selected)

In [ ]:
out_sample_gross_short = indexed_performance(start = end_backtesting, weights = weights_short,
                                             prices = Gross_Price_selected[stocks_invest[stocks_invest.weights < 0].index])

In [ ]:
out_sample_gross_long = indexed_performance(start = end_backtesting, weights = weights_long,
                                            prices = Gross_Price_selected[stocks_invest[stocks_invest.weights > 0].index])

In [ ]:
out_sample_gross = indexed_performance(start = end_backtesting, weights = MSRP_const.x,
                                       prices = Gross_Price_selected)

ombine all returns to data frame (one for in and one for out of sample performance)

In [ ]:
in_sample = pd.DataFrame({"strategy_net": in_sample_net,
                          "strategy_gross": in_sample_gross},
                         index = in_sample_net.index)

In [ ]:
out_sample = pd.DataFrame({"strategy_gross": out_sample_gross,
                           "long_gross": out_sample_gross_long,
                           "short_gross": out_sample_gross_short},
                           index = out_sample_gross.index)

oin benchmark data & calculate indexed performance of benchmark

In [ ]:
in_sample = in_sample.join(benchmark)
out_sample = out_sample.join(benchmark)

In [ ]:
in_sample[["benchmark_gross", "benchmark_net"]] = in_sample[["benchmark_gross", "benchmark_net"]].div(in_sample[["benchmark_gross", "benchmark_net"]].head(1).iloc[0])
out_sample[["benchmark_gross", "benchmark_net"]] = out_sample[["benchmark_gross", "benchmark_net"]].div(out_sample[["benchmark_gross", "benchmark_net"]].head(1).iloc[0])

ave in and out of sample returns

In [ ]:
in_sample.to_csv("files/in_sample.csv")
out_sample.to_csv("files/out_sample.csv")

###########################################################################<br>
                                                                          #<br>
                           #SECTION 3                                     #<br>
       #Calculate in and out of sample risk & performance ratios          #<br>
                                                                          #<br>
###########################################################################

--------------------------------------------------------------------------

In [ ]:
                        #Percent Formatting
#---------------------------------------------------------------------------
#Function that returns a float number rounded to n digits and returns
#it as a string with "%" in the end (ie. in: 0.467, out: 46.7%)

In [ ]:
def to_pct(x, digits):
    out = str(round(x * 100, digits)) + "%"
    return out

--------------------------------------------------------------------------

In [ ]:
                        #Annualized Sharp Ratio
#---------------------------------------------------------------------------

ays = days per year (assumed 250 here)<br>
f_rate = annualized risk free rate<br>
rices for which ratio should be alculated

In [ ]:
def sharp_ratio(price, days, rf_rate):
    ER = price.pct_change().mean()
    SD = price.pct_change().std()
    SR = (ER-rf_rate/days)/SD * m.sqrt(days)
    return round(SR,2)

--------------------------------------------------------------------------

In [ ]:
                        #Yearly volatility
#---------------------------------------------------------------------------

In [ ]:
def yearly_vol(price, days, pct = True):
    vol = m.sqrt(price.pct_change().var() * days)
    if pct:
        return to_pct(vol, 2)
    else:
        return vol

--------------------------------------------------------------------------

In [ ]:
                        #alpha and beta
#---------------------------------------------------------------------------

eriod = period of returns for which regression should be run (eg. "1Y", "1M", "1d")<br>
wargs = optional where only alpha, beta, x or y can be returned form function (param!)

In [ ]:
def alpha_beta(strategy, benchmark, period, pct = True, **kwargs):
    daily_ret_strategy = strategy.pct_change().fillna(0) + 1
    daily_ret_BM = benchmark.pct_change().fillna(0) + 1
    period_ret_strategy = daily_ret_strategy.groupby(pd.Grouper(freq=period)).prod() - 1
    period_ret_BM = daily_ret_BM.groupby(pd.Grouper(freq=period)).prod() -1
    model = LinearRegression().fit(period_ret_BM.to_numpy().reshape((-1,1)),
                                    period_ret_strategy.to_numpy().reshape((-1,1)))
    beta = model.coef_[0][0]
    if pct:
        alpha = to_pct(model.intercept_[0],2)
    else:
        alpha = model.intercept_[0]
    results = {"alpha": alpha,
               "beta": beta,
               "x": period_ret_BM.to_list(),
               "y": period_ret_strategy.to_list()}
    if len(kwargs) == 0:
        return(results)
    else:
        return(results.get(kwargs.get("param")))

--------------------------------------------------------------------------

In [ ]:
                        #Maximum Drawdown
#---------------------------------------------------------------------------

In [ ]:
def maxdd(price, pct = True):
    maxdd = price.diff().cumsum().min()
    if pct:
        return to_pct(maxdd, 2)
    else:
        return maxdd

--------------------------------------------------------------------------

In [ ]:
                        #N Day unfiltered 99% VAR
#---------------------------------------------------------------------------

 as number of days (integer)

In [ ]:
def NDAYVar(price, N, pct = True):
    daily_ret = price.pct_change().fillna(0) + 1
    daily_ret = daily_ret.reset_index(drop = True)
    nday_ret = daily_ret.groupby(daily_ret.index // N).prod() - 1
    VAR = nday_ret.quantile(0.01)
    if pct:
        return to_pct(np.abs(VAR), 2)
    else:
        return np.abs(VAR)

--------------------------------------------------------------------------

In [ ]:
                #Expected N day return or total Return
#---------------------------------------------------------------------------

In [ ]:
def nday_ret(price, N = None, TR = False, pct = True):
    if TR:
        ret = price.tail(1)[0] / price.head(1)[0] -1
    else:
        ret = price.pct_change().mean() * N
    if pct:
        return to_pct(ret, 2)
    else:
        return ret

efine return series for which ratios should be calculated:

In [ ]:
BM_in = in_sample["benchmark_gross"]
BM_out = out_sample["benchmark_gross"]
strategy_in = in_sample["strategy_gross"]
strategy_out = out_sample["strategy_gross"]

ssemble dicts with risk return metrics

In [ ]:
ratios_BM_in = {"Avg. Yearly Return": nday_ret(BM_in, N = 250),
         "Avg. Yearly Sharp Ratio": sharp_ratio(BM_in, days = 250, rf_rate = 0),
         "Max. Drawdown": maxdd(BM_in, "5d"),
         "Alpha (monthly Returns)": "0%",
         "Beta (monthly Returns)": 1,
         "Avg. Ann. Vol": yearly_vol(BM_in, days = 250),
         "5d 99% VAR": NDAYVar(BM_in, N = 5)}

In [ ]:
ratios_BM_out = {"Return YTD": nday_ret(BM_out, TR = True),
         "Avg. Yearly Sharp Ratio": sharp_ratio(BM_out, days = 250, rf_rate = 0),
         "Max. Drawdown": maxdd(BM_out, "5d"),
         "Alpha (weekly Returns)": "0%",
         "Beta (weekly Returns)": 1,
         "Avg. Ann. Vol": yearly_vol(BM_out, days = 250),
         "5d 99% VAR": NDAYVar(BM_out, N = 5)}

In [ ]:
ratios_strategy_in = {"Avg. Yearly Return": nday_ret(strategy_in, N = 250),
               "Avg. Yearly Sharp Ratio": sharp_ratio(strategy_in, days = 250, rf_rate = 0),
               "Max. Drawdown": maxdd(strategy_in, "5d"),
               "Alpha (monthly Returns)": alpha_beta(strategy_in, BM_in, "1M", param = "alpha"),
               "Beta (monthly Returns)": round(alpha_beta(strategy_in, BM_in, "1M", param = "beta"),2),
               "Avg. Ann. Vol": yearly_vol(strategy_in, days = 250),
               "5d 99% VAR": NDAYVar(strategy_in, N = 5)}

In [ ]:
ratios_strategy_out = {"Return YTD": nday_ret(strategy_out, TR = True),
                "Avg. Yearly Sharp Ratio": sharp_ratio(strategy_out, days = 250, rf_rate = 0),
                "Max. Drawdown": maxdd(strategy_out, "5d"),
                "Alpha (weekly Returns)": alpha_beta(strategy_out, BM_out, "1W", param = "alpha"),
                "Beta (weekly Returns)": round(alpha_beta(strategy_out, BM_out, "1W", param = "beta"),2),
                "Avg. Ann. Vol": yearly_vol(strategy_out, days = 250),
                "5d 99% VAR": NDAYVar(strategy_out, N = 5)}

In [ ]:
risk_factors_out = pd.DataFrame({"Benchmark": ratios_BM_in, "Strategy": ratios_strategy_in})
risk_factors_in = pd.DataFrame({"Benchmark": ratios_BM_out, "Strategy": ratios_strategy_out})

In [ ]:
dfi.export(risk_factors_out, "plots/risk_factors_out.png")
dfi.export(risk_factors_in, "plots/risk_factors_in.png")
#---------------------------------------------------------------------------
        #Plot Visual Example for in Sample Alpha/ Beta Calculation
#---------------------------------------------------------------------------
#plot regression line

In [ ]:
params = alpha_beta(strategy_in, BM_in, period = "1M", pct = False)
x = np.arange(min(params.get("x"))-0.1, max(params.get("x"))+0.1, 0.01)
fitted_y = params.get("alpha") + params.get("beta") * x

ssemble plot

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))
plt.scatter(params.get("x"), params.get("y"))
plt.plot(x, fitted_y)
plt.plot(x, x)
plt.title("Monthly Returns Strategy vs. Benchmark (in sample)", size = 25)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.ylabel("Monthly Returns Strategy", size = 15)
plt.xlabel("Monthly Returns Benchmark", size = 15)
plt.axhline(0, color = "black", ls = "--", lw = 1)
plt.axvline(0, color = "black", ls =  "--", lw = 1)
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.legend(["liner fit", "x = y"], prop = {"size": 15})
plt.savefig("plots/in_sample_alphabetaplot.png")

###########################################################################<br>
                                                                          #<br>
                           #SECTION 4                                     #<br>
       #Calculate and plot portfolio characteristics vs. Benchmark        #<br>
                                                                          #<br>
###########################################################################

--------------------------------------------------------------------------

In [ ]:
            #Excess Return Strategy vs. Benchmark Plot
#---------------------------------------------------------------------------

alculate return over out of sample period for all stocks in the benchmark index

In [ ]:
out_sample_all = Gross_Price[Gross_Price.index > end_backtesting]
out_sample_all = out_sample_all.pct_change() + 1
out_sample_all = out_sample_all.fillna(1).cumprod()
out_return_all = out_sample_all.iloc[len(out_sample_all.index)-1].div(out_sample_all.iloc[0]) - 1

alculate excess return contribution as (Weight Strategy - Weigh Index) * Return

In [ ]:
ret_contrib = stocks_index.join(stocks_invest["weights"]).fillna(0)
ret_contrib = ret_contrib[["Name", "weights", "index_weights"]]
ret_contrib["diff_weights"] = ret_contrib["weights"] - ret_contrib["index_weights"]
ret_contrib = ret_contrib.join(pd.DataFrame(out_return_all))
ret_contrib = ret_contrib.rename(columns = {0: "return"})
ret_contrib["diff_return"] = ret_contrib["diff_weights"] * ret_contrib["return"]

lot and save results

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1,1,1)
plt.bar(ret_contrib.Name[0:10], ret_contrib.diff_return[0:10])
plt.bar(ret_contrib.Name[10:39], ret_contrib.diff_return[10:39])
plt.bar(ret_contrib.Name[39:49], ret_contrib.diff_return[39:49])
plt.legend(["Long", "Not Invested", "Short", "Currencies"])
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title("Decomposition of out of Sample excess Returns", size = 22)
plt.ylabel("Excess Return Strategy vs. Benchmark", size = 15)
plt.subplots_adjust(bottom = 0.3)
plt.xticks(rotation=90)
plt.yticks(size = 12)
plt.savefig("plots/excess_return_breakdown.png")

--------------------------------------------------------------------------

In [ ]:
        #Out of Sample Gross Strategy vs Benchmark Performance
#---------------------------------------------------------------------------

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(out_sample[["strategy_gross", "benchmark_gross"]])
plt.legend(out_sample[["strategy_gross", "benchmark_gross"]].columns,
           prop = {"size": 15})
plt.title("Out of Sample Strategy Performance (TR)", size = 20)
plt.ylabel("Cumulative Return", size = 15)
plt.yticks(size = 15)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.xticks(size = 15)
plt.savefig("plots/outofsample_performance.png")

--------------------------------------------------------------------------

In [ ]:
        #In Sample Gross Strategy vs Benchmark Performance
#---------------------------------------------------------------------------

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(in_sample[["strategy_gross", "benchmark_gross"]])
plt.legend(in_sample[["strategy_gross", "benchmark_gross"]].columns,
           prop = {"size": 15})
plt.title("In Sample Strategy Performance", size = 22)
plt.ylabel("Cumulative Return")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(size = 13)
plt.yticks(size = 13)
plt.savefig("plots/insample_performance.png")

--------------------------------------------------------------------------

In [ ]:
                  #In Sample Performance Net vs. Gross
#---------------------------------------------------------------------------

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(in_sample[["strategy_gross", "strategy_net", "benchmark_gross", "benchmark_net"]])
plt.legend(in_sample[["strategy_gross", "strategy_net", "benchmark_gross", "benchmark_net"]].columns,
           prop = {"size": 15})
plt.title("In Sample Strategy Performance (net and gross) Comparison", size = 22)
plt.ylabel("Cumulative Return", size = 15)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(size = 13)
plt.yticks(size = 13)
plt.savefig("plots/insample_performance_netgross.png")

--------------------------------------------------------------------------

In [ ]:
     #Out of Sample Performance Decomposition Long/ Short Portfolio
#---------------------------------------------------------------------------

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(out_sample[["strategy_gross", "benchmark_gross", "long_gross", "short_gross"]])
plt.legend(out_sample[["strategy_gross", "benchmark_gross", "long_gross", "short_gross"]].columns,
           prop = {"size": 15})
plt.title("Decomposition of out of sample performance", size = 22)
plt.ylabel("Cumulative Return", size = 15)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.xticks(size = 13)
plt.yticks(size = 13)
plt.savefig("plots/outofsamlpe_brekdown.png")

--------------------------------------------------------------------------

In [ ]:
    #Stock Weights Index vs. Constrained & Unconstrained Optimization
#---------------------------------------------------------------------------

In [ ]:
width = 0.3
fig, ax = plt.subplots(figsize = (15,10))
ind = np.arange(len(stocks_invest))
ax.barh(ind, stocks_invest.weights, width, label = "Strategy Weights")
ax.barh(ind + width, stocks_invest.index_weights, width, label = "Index Weights")
ax.barh(ind + 2* width, stocks_invest.weights_unconst, width, label = "Unconstrained Strategy Weights")
ax.set(yticks = ind + width, yticklabels = stocks_invest.Name)
ax.legend(prop={'size': 10})
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title("Stock Weights Strategy vs Index", size = 25)
plt.yticks(size = 10)
plt.xticks(size = 15, ticks = np.arange(-0.4, 0.6, 0.1))
plt.savefig("plots/strategy_weights.png")

--------------------------------------------------------------------------

In [ ]:
             #Plot Country Weights Index vs. Strategy
#---------------------------------------------------------------------------

In [ ]:
country_weights_strategy = stocks_invest[["Country", "weights"]].groupby(by = "Country").sum()
country_weights_index = stocks_index[["Country", "index_weights"]].groupby(by = "Country").sum()
country_weights = country_weights_index.join(country_weights_strategy).fillna(0)
country_weights = pd.DataFrame(country_weights, index = country_weights_index.index)

ssemble plot

In [ ]:
width = 0.3
fig, ax = plt.subplots(figsize = (15,10))
ind = np.arange(len(country_weights))
ax.barh(ind, country_weights.weights, width, label = "Strategy Weights")
ax.barh(ind + width, country_weights.index_weights, width, label = "Index Weights")
ax.set(yticks = ind + width, yticklabels = country_weights.index)
ax.legend(prop={'size': 15})
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title("Country Weights Strategy vs Index", size = 25)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.savefig("plots/country_weights.png")

--------------------------------------------------------------------------<br>
lot Currency weights index vs. Strategy<br>
--------------------------------------------------------------------------

In [ ]:
values = stocks_index.join(stocks_invest["weights"]).fillna(0)
currency_weights_strategy = values[["Currency", "weights"]].groupby(by = "Currency").sum()
currency_weights_index = values[["Currency", "index_weights"]].groupby(by = "Currency").sum()

In [ ]:
currency_weights = currency_weights_index.join(currency_weights_strategy).fillna(0)
currency_weights = currency_weights.rename(index = {"GBp": "GBP"})

ssemble plot

In [ ]:
width = 0.4
fig, ax = plt.subplots()
ind = np.arange(len(currency_weights_index))
ax.barh(ind, currency_weights.weights, width, label = "Strategy Weights")
ax.barh(ind + width, currency_weights.index_weights, width, label = "Index Weights")
ax.set(yticks = ind + width, yticklabels = currency_weights.index)
ax.legend(["Strategy", "Index"])
plt.title("Currency Weights Strategy vs Index", size = 20)
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.savefig("plots/curency_comparison.png")

--------------------------------------------------------------------------

In [ ]:
                #Plot sector weights strategy vs. index
#---------------------------------------------------------------------------

In [ ]:
sector_weights_strategy = stocks_invest[["Sector", "weights"]].groupby(by = "Sector").sum()
sector_weights_index = stocks_index[["Sector", "index_weights"]].groupby(by = "Sector").sum()
sector_weights = sector_weights_index.join(sector_weights_strategy)

ssemble plot

In [ ]:
width = 0.4
fig, ax = plt.subplots(figsize=(15, 10))
ind = np.arange(len(sector_weights))
ax.barh(ind, sector_weights.index_weights, width, label = "Strategy Weights")
ax.barh(ind + width, sector_weights.weights, width, label = "Index Weights")
ax.set(yticks = ind + width, yticklabels = sector_weights.index)
ax.legend(prop = {"size": 20})
plt.title("Sector Weights Strategy vs Index", size = 25)
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.subplots_adjust(left = 0.2)
plt.savefig("plots/sector_weights.png")

--------------------------------------------------------------------------

In [ ]:
    #Plot performance Nasdaq100 vs. MSCI World vs. FTSE AW-HighDivYield
#---------------------------------------------------------------------------

ownload prices and clean data

In [ ]:
prices = yf.download(tickers = ["VHYL.AS", "EXXT.DE", "IWDA.AS"], start = "2022-01-01", end = dt.date.today())
prices = prices["Adj Close"]
prices = prices.div(prices.iloc[0])
prices = prices.rename(columns = {"EXXT.DE": "Nasdaq 100", "IWDA.AS": "MSCI World", "VHYL.AS": "FTSE All World High Dividend"})

ssemble plot

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(prices)
plt.title("High Dividend Stocks vs. Technology Stocks vs. Total Market", size = 25)
plt.legend(prices.columns,  prop={'size': 15})
plt.ylabel("Cumulative Return", size = 15)
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.savefig("comparison.png")

--------------------------------------------------------------------------

In [ ]:
    #calculate change of correlation  matrix in vs out of sample
#---------------------------------------------------------------------------

In [ ]:
out_sample_cor = Gross_Price[Gross_Price.index > end_backtesting].pct_change().corr()
in_sample_cor =  Gross_Price[Gross_Price.index < end_backtesting].pct_change().corr()

In [ ]:
rel_cor = out_sample_cor / in_sample_cor
pd.DataFrame(rel_cor).to_csv("files/correlation_change.csv")

--------------------------------------------------------------------------

In [ ]:
  #calculate performance ratios for short and long portfolios & Benchmark
#---------------------------------------------------------------------------


<br>
we only include stocks which contain a forward,trailing PE ratio &<br>
PB Ratio to calculate weighted values. <br>
Further, as described by iShares,<br>
we restrict the maximum value of PE ratios to 60 and of PB ratios to 25. <br>


In [ ]:
stocks_short = stocks_invest[stocks_invest.weights <0]
stocks_long = stocks_invest[stocks_invest.weights > 0]

ividend yield

In [ ]:
div_yield_long = (stocks_long.weights / stocks_long.weights.sum() * stocks_long.Yield).sum()
div_yield_short = (stocks_short.weights.abs()/ stocks_short.weights.abs().sum() * stocks_short.Yield).sum()
div_yield_index = (stocks_index.index_weights * stocks_index.Yield).sum()

railing PE Ratio, forward PE, PB Ratio

et all stocks and join strategy weights

In [ ]:
index = stocks_index.join(stocks_invest[["weights"]])

. remove stocks where ratios are not given

In [ ]:
index = index[index.Trailing_PE.notna()]
index = index[index.Forward_PE.notna()]

. set PE ratios above 60 to 60, set PB ratios above 25 to 25

In [ ]:
index.Trailing_PE = np.where(index.Trailing_PE > 60, 60, index.Trailing_PE)
index.Forward_PE = np.where(index.Forward_PE > 60, 60, index.Forward_PE)
index.PB_Ratio = np.where(index.PB_Ratio > 25, 25, index.PB_Ratio)

. separate long and short stocks from strategy to calculate ratios separately

In [ ]:
short = index.loc[stocks_short.index]
long = index.loc[stocks_long.index]

alculate Trailing PE

In [ ]:
PE_long = (long.weights / long.weights.sum() * long.Trailing_PE).sum()
PE_short = (short.weights / short.weights.sum() * short.Trailing_PE).sum()
PE_index = (index.index_weights * index.Trailing_PE).sum()

alculate forward PE

In [ ]:
PE_fwd_long = (long.weights / long.weights.sum() * long.Forward_PE).sum()
PE_fwd_short = (short.weights / short.weights.sum() * short.Forward_PE).sum()
PE_fwd_index = (index.index_weights * index.Forward_PE).sum()

alculate PB Ratio

In [ ]:
PB_long = (long.weights / long.weights.sum() * long.PB_Ratio).sum()
PB_short = (short.weights / short.weights.sum() * short.PB_Ratio).sum()
PB_index = (index.index_weights * index.PB_Ratio).sum()

ssemble metrics dataframe

In [ ]:
ratios_short = {"Yield": to_pct(div_yield_short,2),
                "Price_Book": round(PB_short,2),
                "Trailing_PE": round(PE_short,2),
                "Forward_PE": round(PE_fwd_short,2)}

In [ ]:
ratios_long = {"Yield": to_pct(div_yield_long,2),
               "Price_Book": round(PB_long,2),
               "Trailing_PE": round(PE_long,2),
               "Forward_PE": round(PE_fwd_long,2)}

In [ ]:
ratios_index = {"Yield": to_pct(div_yield_index,2),
                "Price_Book": round(PB_index,2),
                "Trailing_PE": round(PE_index,2),
                "Forward_PE": round(PE_fwd_index,2)}

In [ ]:
ratios_table = pd.DataFrame({"Portfolio Short":ratios_short, "Portfolio Long": ratios_long, "Index": ratios_index})

xport table

In [ ]:
dfi.export(ratios_table, "plots/portfolio_characteristics.png")